<a href="https://colab.research.google.com/github/mfathul21/loan-prediction/blob/main/04_Loan_Prediction_Based_on_Customers_Behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loan Prediction Based on Customers Behaviour

## Domain Project

### Background

## Business Understanding

## Data Understanding

Data yang Anda gunakan pada proyek kali ini adalah ["Loan Prediction Based on Customers Behaviour"](https://www.kaggle.com/datasets/subhamjain/loan-prediction-based-on-customer-behavior?select=Training+Data.csv) yang diunduh dari Kaggle API. Untuk tahap training, dapat dilakukan hanya dari file "Training Data.csv" yang akan displit kembali untuk menjadi data train dan test pada tahap modelling. Selain itu, dataset ini juga cukup bersih sehingga tidak terlalu banyak memerlukan proses data cleaning.

Dataset ini memiliki 252000 baris dengan 13 fitur, yang terdiri fitur non-numerik seperti Married/Single, House_Ownership, Car_Ownership, Profession, CITY, dan STATE, serta fitur numerik seperti Id, Income, Age, Experience, CURRENT_JOB_YRS, CURRENT_HOUSE_YRS, dan Risk_Flag. Fitur-fitur ini adalah fitur yang akan Anda gunakan dalam menemukan pola pada data untuk memprediksi fitur target yaitu Risk_Flag.

## Data Loading

### Download datasets from Kaggle to Google Colab

In [1]:
!pip install nbformat==5.10.4
!pip install nbconvert==7.16.1
!pip install kaggle -q
!pip install shap -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.5.4
    Uninstalling nbconvert-6.5.4:
      Successfully uninstalled nbconvert-6.5.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 8.9 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muhammadfathul","key":"86f7af68314a74218ed685350e322691"}'}

In [3]:
import json
import os

# Check if the Kaggle API configuration directory exists, and if not, create it.
if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')

# Move the uploaded Kaggle JSON file to the configuration directory.
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')

# Set the permissions for the Kaggle JSON file.
os.chmod('/root/.kaggle/kaggle.json', 600)

In [4]:
!kaggle datasets download -d subhamjain/loan-prediction-based-on-customer-behavior

Dataset URL: https://www.kaggle.com/datasets/subhamjain/loan-prediction-based-on-customer-behavior
License(s): other
  0% 0.00/5.15M [00:00<?, ?B/s]
100% 5.15M/5.15M [00:00<00:00, 59.1MB/s]


In [5]:
!unzip -q loan-prediction-based-on-customer-behavior.zip

### Import Datasets

In [6]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('Training Data.csv')
df.sample(5)

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
198449,198450,3392494,28,6,single,rented,yes,Secretary,Jamshedpur,Jharkhand,6,10,0
19799,19800,3529385,73,18,single,owned,no,Civil_servant,Aurangabad,Maharashtra,14,11,0
106850,106851,7968958,36,15,single,rented,yes,Police_officer,Gaya,Bihar,3,10,0
137222,137223,1369446,59,3,single,rented,no,Police_officer,Bathinda,Punjab,3,11,1
128928,128929,5510241,78,14,single,rented,yes,Psychologist,Patiala,Punjab,11,11,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Id                 252000 non-null  int64 
 1   Income             252000 non-null  int64 
 2   Age                252000 non-null  int64 
 3   Experience         252000 non-null  int64 
 4   Married/Single     252000 non-null  object
 5   House_Ownership    252000 non-null  object
 6   Car_Ownership      252000 non-null  object
 7   Profession         252000 non-null  object
 8   CITY               252000 non-null  object
 9   STATE              252000 non-null  object
 10  CURRENT_JOB_YRS    252000 non-null  int64 
 11  CURRENT_HOUSE_YRS  252000 non-null  int64 
 12  Risk_Flag          252000 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 25.0+ MB


In [9]:
df.select_dtypes([int, float]).columns

Index(['Id', 'Income', 'Age', 'Experience', 'CURRENT_JOB_YRS',
       'CURRENT_HOUSE_YRS', 'Risk_Flag'],
      dtype='object')

In [10]:
df.select_dtypes([object]).columns

Index(['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession',
       'CITY', 'STATE'],
      dtype='object')